In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # number of independent sequences will we process in parallel
block_size = 32 # maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

In [18]:
# This is the dataset to train on.
!wget https://www.gutenberg.org/files/132/132-0.txt

--2025-08-02 18:15:23--  https://www.gutenberg.org/files/132/132-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322409 (315K) [text/plain]
Saving to: ‘132-0.txt.1’

132-0.txt.1         100%[===================>] 314.85K   662KB/s    in 0.5s    

2025-08-02 18:15:24 (662 KB/s) - ‘132-0.txt.1’ saved [322409/322409]



In [19]:
# read dataset
with open('132-0.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [20]:
print("the dataset has this many characters: ", len(text))

the dataset has this many characters:  312936


In [21]:
print("first 1000 characters of dataset: ", text[:1000])

first 1000 characters of dataset:  *** START OF THE PROJECT GUTENBERG EBOOK 132 ***
Sun Tzŭ
on
The Art of War

THE OLDEST MILITARY TREATISE IN THE WORLD
Translated from the Chinese with Introduction and Critical Notes

BY
LIONEL GILES, M.A.

Assistant in the Department of Oriental Printed Books and MSS.
in the British Museum




1910



To my brother
Captain Valentine Giles, R.G.
in the hope that
a work 2400 years old
may yet contain lessons worth consideration
by the soldier of today
this translation
is affectionately dedicated.



Contents


 Preface to the Project Gutenberg Etext
 Preface by Lionel Giles
 INTRODUCTION
 Sun Wu and his Book
 The Text of Sun Tzŭ
 The Commentators
 Appreciations of Sun Tzŭ
 Apologies for War
 Bibliography
 Chapter I. Laying plans
 Chapter II. Waging War
 Chapter III. Attack by Stratagem
 Chapter IV. Tactical Dispositions
 Chapter V. Energy
 Chapter VI. Weak Points and Strong
 Chapter VII Manœuvring
 Chapter VIII. Variation of Tactics
 Chapter IX. The Ar

In [22]:
# list of all unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"&()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz§àâèéêôüŒœŭ—‘’“”…
97


In [23]:
# now we create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: change the string input into list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: change the list of integers and output a string


In [24]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [25]:
def get_batch(split):
    # this function generates a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [26]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [27]:
class Head(nn.Module):
    #self attention

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [28]:
class MultiHeadAttention(nn.Module):
    # multiple heads of self-attention in parallel

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [29]:
class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [30]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [31]:
class BigramLanguageModel(nn.Module):

    def __init__(self): # each token directly reads off the logits for the next token from a lookup table
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [32]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print("number of parameters in the model", sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}") # show loss in intervals in train and val sets

    xb, yb = get_batch('train') # sample batch

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

number of parameters in the model 0.213857 M parameters
step 0: train loss 4.7945, val loss 4.7967
step 100: train loss 2.7248, val loss 2.7255
step 200: train loss 2.5803, val loss 2.5729
step 300: train loss 2.4792, val loss 2.4724
step 400: train loss 2.4094, val loss 2.4047
step 500: train loss 2.3286, val loss 2.3438
step 600: train loss 2.2835, val loss 2.2841
step 700: train loss 2.2075, val loss 2.2186
step 800: train loss 2.1529, val loss 2.1559


KeyboardInterrupt: 

In [ ]:
# generate using the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))